# Indicators
## How to compute indicators
The syntax for declaring the indicators used and for computing these indicators is provided below. Basically, all supported indicators are shown. Note that before computing indicator you must first specify it in the `indicators` attribute of the trading strategy.

In [3]:
from backintime import FuturesStrategy
from backintime.timeframes import Timeframes as tf
from backintime.indicator_params import *
from backintime.analyser.indicators.constants import CLOSE

class DummyStrategy(FuturesStrategy):
    # You need to declare input series for indicators in advance
    indicators = {
        ADX(tf.M1, period=14),
        ATR(tf.M1, period=14),
        BBANDS(tf.M1, CLOSE, period=20),
        DMI(tf.M1, period=14),
        # It is possible to use the same indicators but with different params
        EMA(tf.M1, CLOSE, period=9),
        EMA(tf.M1, CLOSE, period=20),
        MACD(tf.M1, fastperiod=12, slowperiod=26, signalperiod=9),
        PIVOT(tf.M1, period=15),
        RSI(tf.M1, period=14),
        SMA(tf.M1, CLOSE, period=9),
        KELTNER_CHANNEL(tf.M1, period=20)
    }
    def tick(self):
        # self.analyser is used to compute indicators
        adx = self.analyser.adx(tf.M1, period=14)
        atr = self.analyser.atr(tf.M1, period=14)
        bbands = self.analyser.bbands(tf.M1, CLOSE, period=20)
        dmi = self.analyser.dmi(tf.M1, period=14)
        short_ema = self.analyser.ema(tf.M1, CLOSE, period=9)
        long_ema = self.analyser.ema(tf.M1, CLOSE, period=20)
        macd = self.analyser.macd(tf.M1, 12, 26, 9)
        pivot = self.analyser.pivot(tf.M1, period=15)
        pivot_classic = self.analyser.pivot_classic(tf.M1, period=15)
        pivot_fib = self.analyser.pivot_fib(tf.M1, period=15)
        rsi = self.analyser.rsi(tf.M1, period=14)
        sma = self.analyser.sma(tf.M1, CLOSE, period=9)
        kc = self.analyser.keltner_channel(tf.M1, OPEN, period=20, atr_period=14, multiplier=2)

In order to actually get the results we need to run backtesting. Here's the full code with imports, feed and other necessary setup. Here we are computing only one indicator, MACD, for simplicity.

In [5]:
import os
from datetime import datetime
from backintime import FuturesStrategy, run_backtest
from backintime.trading_strategy import AbstractStrategyFactory
from backintime.timeframes import Timeframes as tf
from backintime.data.csv import CSVCandlesFactory, CSVCandlesSchema
from backintime.indicator_params import MACD
from backintime.utils import PREFETCH_SINCE

import pandas as pd  # going to save the results in DataFrame
macd_data = pd.DataFrame(columns=['upd_time', 'macd', 'signal', 'hist'])

class DummyStrategy(FuturesStrategy):
    candle_timeframes = { tf.M1 }
    indicators = { MACD(tf.M1, fastperiod=12, slowperiod=26, signalperiod=9) }

    def tick(self):
        m1 = self.candles.get(tf.M1)
        macd = self.analyser.macd(tf.M1, 12, 26, 9)
        macd = macd[-1] # we actually only need the last (most recent) value here
        macd_data.loc[len(macd_data)] = [m1.close_time, macd.macd, macd.signal, macd.hist]

class DummyFactory(AbstractStrategyFactory):
    def create(self, broker, analyser, candles):
        return DummyStrategy(broker, analyser, candles)

# Locate our input file
datadir = os.path.join(os.path.abspath(''), 'data')
datafile = os.path.join(datadir, 'MNQ_week_1min_continuous_UNadjusted_20240425_fixed.csv')
# Declare schema of the input file
schema = CSVCandlesSchema(open_time=0, open=1, high=2,
                          low=3, close=4, volume=5, close_time=6)
# Declare feed
feed = CSVCandlesFactory(datafile, 'MNQUSD', tf.M1, 
                         delimiter=',', schema=schema,
                         timezone='America/New_York')

since = datetime.fromisoformat("2024-03-10 18:00:00-04:00")
until = datetime.fromisoformat("2024-03-10 23:00:00-04:00")

run_backtest(DummyStrategy, DummyFactory(),
             feed, 10_000, since, until, None,
             prefetch_option=PREFETCH_SINCE)

print(macd_data.head())

INFO:backintime:Start prefetching...
INFO:backintime:count: 234
INFO:backintime:since: 2024-03-10 18:00:00-04:00
INFO:backintime:until: 2024-03-10 21:54:00-04:00
INFO:backintime:Prefetching is done
INFO:backintime:Start backtesting...
INFO:backintime:Backtesting is done


                          upd_time      macd    signal      hist
0 2024-03-10 21:54:59.999000-04:00 -1.149881  1.066266 -2.216147
1 2024-03-10 21:55:59.999000-04:00 -1.453491  0.562315 -2.015806
2 2024-03-10 21:56:59.999000-04:00 -1.674798  0.114893 -1.789690
3 2024-03-10 21:57:59.999000-04:00 -1.609727 -0.230031 -1.379696
4 2024-03-10 21:58:59.999000-04:00 -1.919319 -0.567888 -1.351430


## What `indicator_params` really are?
`MACD`, `EMA` and other indicators names listed in the `indicators` attribute are actually just functions. They are aliases for more verbose counterparts: `macd_params`, `ema_params` etc. This functions return _a specification of input time series_ required to reliably compute corresponding indicators. For instance, `EMA` (which is the same as `ema_params`) accepts three arguments: timeframe, candle property (one of OHLC) and period. It return a list containing hints for the engine: what data it needs to pull in advance (what timeframe, what price) and how much of the data it needs to store (period argument directly controls this). Here's the code.
```py
def ema_params(timeframe: Timeframes, 
               candle_property: CandleProperties = CLOSE,
               period: int = 9) -> t.Tuple[IndicatorParam]:
    """Get list of EMA params."""
    return (
        IndicatorParam(timeframe=timeframe, 
                       candle_property=candle_property, 
                       quantity=period**2),
    )
```
I understand that calling this 'params' in the code might be misleading here but this is how it is now. The parameters for these function does not necessarily the same as the parameters for actual computing (i.e. one that you would do with `self.analyser`). Consider this example: here's the code for Keltner Channel params
```py
def keltner_channel_params(timeframe: Timeframes,
                           period: int = 20) -> t.Tuple[IndicatorParam]:
    """Get list of Keltner Channel params."""
    return (
        IndicatorParam(timeframe, OPEN, period**2),
        IndicatorParam(timeframe, HIGH, period**2),
        IndicatorParam(timeframe, LOW, period**2),
        IndicatorParam(timeframe, CLOSE, period**2)
    )
```
And this is the signature to compute the indicator
```py
def keltner_channel(self, 
                    timeframe: Timeframes,
                    candle_property: CandleProperties = CLOSE,
                    period: int = 20,
                    atr_period: int = 10,
                    multiplier: int = 2) -> KeltnerChannelResultSequence:
    pass
```
They have different arguments (keltner_channel has a bit more). This is because not all of its params are relevant to identify the proprties of its input time series. Like there's `multiplier` param, but it only affects computing. Having multiplier set to 4 does not mean we would need twice as much data. Conversely, period param is important since the amount of data needed depends on it, so it exists in the list of `keltner_channel_params` arguments, too.

## How it works
Indicators computing are implemented in the `Analyser` class that is located in `src/backintime/analyser/analyser.py`. More precisely there's a module for each indicator supported currently and `Analyser` uses this modules to actually compute anything. But `Analyser` is designed to provide more user friendly interface and that's why it is exposed to the user code. `Analyser` uses auxillary class, `AnalyserBuffer` to store market data. `AnalyserBuffer` is built on top of round-buffer-like structures (python `collections.deque` is used at the moment) and stores only the required amount of the most recent data. Here's the demo that shows an idea how these classes are used by the engine.

In [4]:
from decimal import Decimal
from datetime import datetime, timedelta

from backintime.candles import InputCandle
from backintime.indicator_params import SMA
from backintime.analyser.analyser import Analyser, AnalyserBuffer
from backintime.analyser.indicators.constants import CLOSE
from backintime.timeframes import Timeframes as tf

start_time = datetime.fromisoformat('2020-01-01 00:00+00:00')
sample_candles = [    # our sample data
    InputCandle(open_time=start_time,
                open=Decimal(1000),
                high=Decimal(1050),
                low=Decimal(950),
                close=Decimal(1050),
                volume=Decimal(100),
                close_time=start_time + timedelta(seconds=59)),
    
    InputCandle(open_time=start_time + timedelta(minutes=1),
                open=Decimal(1100),
                high=Decimal(1150),
                low=Decimal(1050),
                close=Decimal(1150),
                volume=Decimal(100),
                close_time=start_time + timedelta(minutes=1, seconds=59)),

    InputCandle(open_time=start_time + timedelta(minutes=2),
                open=Decimal(1200),
                high=Decimal(1250),
                low=Decimal(1150),
                close=Decimal(1150),
                volume=Decimal(100),
                close_time=start_time + timedelta(minutes=2, seconds=59)),

    InputCandle(open_time=start_time + timedelta(minutes=3),
                open=Decimal(1300),
                high=Decimal(1350),
                low=Decimal(1250),
                close=Decimal(1250),
                volume=Decimal(100),
                close_time=start_time + timedelta(minutes=3, seconds=59)),
    
    InputCandle(open_time=start_time + timedelta(minutes=4),
                open=Decimal(1400),
                high=Decimal(1450),
                low=Decimal(1350),
                close=Decimal(1450),
                volume=Decimal(100),
                close_time=start_time + timedelta(minutes=4, seconds=59)),
]
# Initialize AnalyserBuffer
analyser_buffer = AnalyserBuffer(start_time)
for item in SMA(tf.M1, CLOSE, period=5):   # there's going to be only one item but that's not the case for many other indicators
    analyser_buffer.reserve(item.timeframe, item.candle_property, item.quantity)  # tell buffer what data/what amount it needs to store
# Initialize Analyser
analyser = Analyser(analyser_buffer)
# Push new values
for candle in sample_candles:
    analyser_buffer.update(candle)
# Retrieve indicator value
sma = analyser.sma(tf.M1, CLOSE, period=5)
print(f"SMA series\n{sma}")
print(f"Last SMA: {sma[-1]}")

SMA series
[  nan   nan   nan   nan 1210.]
Last SMA: 1210.0
